Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/NotebookVM/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-with-automated-machine-learning-step.png)

# Azure Machine Learning Pipeline with AutoMLStep (Udacity Course 2)
This notebook demonstrates the use of AutoMLStep in Azure Machine Learning Pipeline.

## Introduction
In this example we showcase how you can use AzureML Dataset to load data for AutoML via AML Pipeline. 

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you have executed the [configuration](https://aka.ms/pl-config) before running this notebook.

In this notebook you will learn how to:
1. Create an `Experiment` in an existing `Workspace`.
2. Create or Attach existing AmlCompute to a workspace.
3. Define data loading in a `TabularDataset`.
4. Configure AutoML using `AutoMLConfig`.
5. Use AutoMLStep
6. Train the model using AmlCompute
7. Explore the results.
8. Test the best fitted model.

## Azure Machine Learning and Pipeline SDK-specific imports

In [1]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.38.0


## Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-189502
aml-quickstarts-189502
southcentralus
5a4ab2ba-6c51-4805-8155-58759ad589d8


## Create an Azure ML experiment
Let's create an experiment named "automlstep-classification" and a folder to hold the training scripts. The script runs will be recorded under the experiment in Azure.

The best practice is to use separate folders for scripts and its dependent files for each step and specify that folder as the `source_directory` for the step. This helps reduce the size of the snapshot created for the step (only the specific folder is snapshotted). Since changes in any files in the `source_directory` would trigger a re-upload of the snapshot, this helps keep the reuse of the step when there are no changes in the `source_directory` of the step.

*Udacity Note:* There is no need to create an Azure ML experiment, this needs to re-use the experiment that was already created


In [3]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'aml-bank-marketing-exp'
project_folder = './pipeline-project'


experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
aml-bank-marketing-exp,quick-starts-ws-189502,Link to Azure Machine Learning studio,Link to Documentation


### Create or Attach an AmlCompute cluster
You will need to create a [compute target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) for your AutoML run. In this tutorial, you get the default `AmlCompute` as your training compute resource.

**Udacity Note** There is no need to create a new compute target, it can re-use the previous cluster

In [4]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "compute-cluster1"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Data

**Udacity note:** Make sure the `key` is the same name as the dataset that is uploaded, and that the description matches. If it is hard to find or unknown, loop over the `ws.datasets.keys()` and `print()` them.
If it *isn't* found because it was deleted, it can be recreated with the link that has the CSV 

In [5]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '5a4ab2ba-6c51-4805-8155-58759ad589d8'
resource_group = 'aml-quickstarts-189502'
workspace_name = 'quick-starts-ws-189502'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='aml-bank-maerketing')
dataset.to_pandas_dataframe()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32945,56,housemaid,married,basic.4y,no,no,yes,cellular,jul,mon,...,1,999,0,nonexistent,1.4,93.918,-42.7,4.960,5228.1,no
32946,37,management,married,university.degree,no,no,yes,cellular,jul,fri,...,7,999,0,nonexistent,1.4,93.918,-42.7,4.957,5228.1,no
32947,26,admin.,single,university.degree,no,no,no,cellular,may,tue,...,4,999,1,failure,-1.8,92.893,-46.2,1.266,5099.1,no
32948,31,blue-collar,single,basic.9y,no,no,no,cellular,apr,mon,...,1,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,no


### Review the Dataset Result

You can peek the result of a TabularDataset at any range using `skip(i)` and `take(j).to_pandas_dataframe()`. Doing so evaluates only `j` records for all the steps in the TabularDataset, which makes it fast even against large datasets.

`TabularDataset` objects are composed of a list of transformation steps (optional).

In [6]:
dataset.take(5).to_pandas_dataframe()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


## Train
This creates a general AutoML settings object.
**Udacity notes:** These inputs must match what was used when training in the portal. `label_column_name` has to be `y` for example.

In [7]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="y",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

#### Create Pipeline and AutoMLStep

You can define outputs for the AutoMLStep using TrainingOutput.

In [8]:
from azureml.pipeline.core import PipelineData, TrainingOutput

ds = ws.get_default_datastore()
metrics_output_name = 'metrics_output'
best_model_output_name = 'best_model_output'

metrics_data = PipelineData(name='metrics_data',
                           datastore=ds,
                           pipeline_output_name=metrics_output_name,
                           training_output=TrainingOutput(type='Metrics'))
model_data = PipelineData(name='model_data',
                           datastore=ds,
                           pipeline_output_name=best_model_output_name,
                           training_output=TrainingOutput(type='Model'))

Create an AutoMLStep.

In [9]:
automl_step = AutoMLStep(
    name='automl_module',
    automl_config=automl_config,
    outputs=[metrics_data, model_data],
    allow_reuse=True)

In [10]:
from azureml.pipeline.core import Pipeline
pipeline = Pipeline(
    description="pipeline_with_automlstep",
    workspace=ws,    
    steps=[automl_step])

In [11]:
pipeline_run = experiment.submit(pipeline)

Created step automl_module [13eb2eda][700d6487-0555-4036-b174-a46a9e560cfb], (This step will run and generate new outputs)
Submitted PipelineRun 6f86276d-c29a-4dbd-98dc-538743df3ab2
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/6f86276d-c29a-4dbd-98dc-538743df3ab2?wsid=/subscriptions/5a4ab2ba-6c51-4805-8155-58759ad589d8/resourcegroups/aml-quickstarts-189502/workspaces/quick-starts-ws-189502&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254


In [12]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [13]:
pipeline_run.wait_for_completion()

PipelineRunId: 6f86276d-c29a-4dbd-98dc-538743df3ab2
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/6f86276d-c29a-4dbd-98dc-538743df3ab2?wsid=/subscriptions/5a4ab2ba-6c51-4805-8155-58759ad589d8/resourcegroups/aml-quickstarts-189502/workspaces/quick-starts-ws-189502&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: a37b57e0-b4cf-455d-aa34-7b6f1a304ba1
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/a37b57e0-b4cf-455d-aa34-7b6f1a304ba1?wsid=/subscriptions/5a4ab2ba-6c51-4805-8155-58759ad589d8/resourcegroups/aml-quickstarts-189502/workspaces/quick-starts-ws-189502&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254
StepRun( automl_module ) Status: Running

StepRun(automl_module) Execution Summary
StepRun( automl_module ) Status: Finished

Warnings:
No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False 

'Finished'

## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [14]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
num_file_downloaded = metrics_output.download('.', show_progress=True)

Downloaded azureml/a37b57e0-b4cf-455d-aa34-7b6f1a304ba1/metrics_data, 1 files out of an estimated total of 1


In [15]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

,a37b57e0-b4cf-455d-aa34-7b6f1a304ba1_5,a37b57e0-b4cf-455d-aa34-7b6f1a304ba1_8,a37b57e0-b4cf-455d-aa34-7b6f1a304ba1_6,a37b57e0-b4cf-455d-aa34-7b6f1a304ba1_7,a37b57e0-b4cf-455d-aa34-7b6f1a304ba1_10,a37b57e0-b4cf-455d-aa34-7b6f1a304ba1_9,a37b57e0-b4cf-455d-aa34-7b6f1a304ba1_11,a37b57e0-b4cf-455d-aa34-7b6f1a304ba1_12,a37b57e0-b4cf-455d-aa34-7b6f1a304ba1_13,a37b57e0-b4cf-455d-aa34-7b6f1a304ba1_15,...,a37b57e0-b4cf-455d-aa34-7b6f1a304ba1_4,a37b57e0-b4cf-455d-aa34-7b6f1a304ba1_29,a37b57e0-b4cf-455d-aa34-7b6f1a304ba1_31,a37b57e0-b4cf-455d-aa34-7b6f1a304ba1_24,a37b57e0-b4cf-455d-aa34-7b6f1a304ba1_3,a37b57e0-b4cf-455d-aa34-7b6f1a304ba1_28,a37b57e0-b4cf-455d-aa34-7b6f1a304ba1_2,a37b57e0-b4cf-455d-aa34-7b6f1a304ba1_0,a37b57e0-b4cf-455d-aa34-7b6f1a304ba1_1,a37b57e0-b4cf-455d-aa34-7b6f1a304ba1_25
recall_score_weighted,[0.9095599393019727],[0.9150227617602428],[0.7960546282245827],[0.9050075872534142],[0.9071320182094081],[0.7566009104704097],[0.7666160849772382],[0.9138088012139606],[0.8880121396054628],[0.9062215477996965],...,[0.9028831562974203],[0.9113808801213961],[0.9116843702579667],[0.9092564491654022],[0.891350531107739],[0.9125948406676783],[0.8986342943854325],[0.9116843702579667],[0.9071320182094081],[0.9053110773899848]
average_precision_score_micro,[0.9787148095664376],[0.9796972056918486],[0.9119673700979228],[0.9741632043235319],[0.9749456369318957],[0.8014668312769941],[0.8064515683641996],[0.9796736019290663],[0.9543149331348836],[0.9785521915837849],...,[0.9740908987364524],[0.9804991765966256],[0.9801291032309792],[0.9788822950863237],[0.9578402356356086],[0.9804868320844866],[0.9667031506254842],[0.9806603102489483],[0.9791945367231853],[0.9773215584829349]
f1_score_macro,[0.7014585481893716],[0.7417439811175828],[0.6886964345967832],[0.6749695323823424],[0.7042835724240846],[0.6260247351623556],[0.6321861888897324],[0.7582948132750074],[0.4703423886834914],[0.6494711021216164],...,[0.6485586371114613],[0.7599652324555626],[0.7617201565893288],[0.716038415899001],[0.5257994127616603],[0.7638698118841445],[0.6276085302899465],[0.7653697272147331],[0.7416848907681176],[0.716504738709165]
AUC_macro,[0.937258612162335],[0.9398820406522588],[0.9242618741976893],[0.9214624699220335],[0.9229564117240625],[0.8357946788627149],[0.8338742273273724],[0.9405160165750666],[0.8413995076382752],[0.9435756797759365],...,[0.9222052729754913],[0.9445453989741537],[0.9430672023740059],[0.9378610976813802],[0.8586043823527777],[0.944476861036553],[0.887650111976171],[0.9450464668693166],[0.9392346349984347],[0.9311573464333414]
average_precision_score_macro,[0.8097168063826393],[0.8196595960776589],[0.7774309003710416],[0.7895864147291611],[0.7798530705146514],[0.7204025001949677],[0.7210371386441244],[0.8136182410340502],[0.7104022926576434],[0.8172609410184801],...,[0.769196899235336],[0.8161916881353775],[0.8132205271419203],[0.810299029799012],[0.7172916262038993],[0.8151648767326019],[0.7253016633376068],[0.8151093723721079],[0.8065229883244922],[0.7983443889706254]
precision_score_micro,[0.9095599393019727],[0.9150227617602428],[0.7960546282245827],[0.9050075872534142],[0.9071320182094081],[0.7566009104704097],[0.7666160849772382],[0.9138088012139606],[0.8880121396054628],[0.9062215477996965],...,[0.9028831562974203],[0.9113808801213961],[0.9116843702579667],[0.9092564491654022],[0.891350531107739],[0.9125948406676783],[0.8986342943854325],[0.9116843702579667],[0.9071320182094081],[0.9053110773899848]
log_loss,[0.3295607672860524],[0.3243792187377307],[0.3875688528362605],[0.3408271134784256],[0.21264872995600134],[0.5916179666117642],[0.553579790149044],[0.21143958465162815],[0.2850386342092603],[0.2127056598602643],...,[0.22229585606904445],[0.1799133147004271],[0.182237347815391],[0.32741181617971277],[0.26905259370191814],[0.18020078571133624],[0.251737541051616],[0.17775706110025447],[0.18743634958563793],[0.19904580058680338]
balanced_accuracy,[0.6577808156755525],[0.7023013001832],[0.8555641691071729],[0.6339032170226009],[0.

### Retrieve the Best Model

In [16]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

Downloaded azureml/a37b57e0-b4cf-455d-aa34-7b6f1a304ba1/model_data, 1 files out of an estimated total of 1


In [17]:
!pip install --upgrade azureml-sdk[automl]

Requirement already up-to-date: azureml-sdk[automl] in /anaconda/envs/azureml_py36/lib/python3.6/site-packages (1.39.0)


In [18]:
!pip list

Package                                 Version
--------------------------------------- -------------------
absl-py                                 0.15.0
adal                                    1.2.7
alembic                                 1.7.6
ansiwrap                                0.8.4
antlr4-python3-runtime                  4.7.2
applicationinsights                     0.11.10
arch                                    4.14
argcomplete                             1.12.3
argon2-cffi                             21.3.0
argon2-cffi-bindings                    21.2.0
astor                                   0.8.1
astunparse                              1.6.3
attrs                                   21.4.0
autokeras                               1.0.16
azure-appconfiguration                  1.1.1
azure-batch                             11.0.0
azure-cli                               2.33.0
azure-cli-core                          2.33.0
azure-cli-telemetry                     1.0.6
azure-co

In [40]:
pip install --upgrade azureml-sdk[notebook,automl]

  Using cached azureml_sdk-1.39.0-py3-none-any.whl (4.5 kB)
  Using cached azureml_pipeline-1.39.0-py3-none-any.whl (3.7 kB)
  Using cached azureml_train_core-1.39.0-py3-none-any.whl (8.6 MB)
  Using cached azureml_core-1.39.0.post1-py3-none-any.whl (2.5 MB)
  Using cached azureml_train_automl_client-1.39.0-py3-none-any.whl (136 kB)
  Using cached azureml_dataset_runtime-1.39.0-py3-none-any.whl (3.5 kB)
  Using cached azureml_train_automl-1.39.0-py3-none-any.whl (3.5 kB)
  Using cached azureml_pipeline_steps-1.39.0-py3-none-any.whl (70 kB)
  Using cached azureml_pipeline_core-1.39.0-py3-none-any.whl (313 kB)
  Using cached azureml_telemetry-1.39.0-py3-none-any.whl (31 kB)
  Using cached azureml_train_restclients_hyperdrive-1.39.0-py3-none-any.whl (19 kB)
  Using cached knack-0.9.0-py3-none-any.whl (59 kB)
  Using cached azureml_automl_core-1.39.1-py3-none-any.whl (228 kB)
  Using cached azureml_dataprep-2.27.1-py3-none-any.whl (39.4 MB)
  Using cached azureml_train_automl_runtime-1.39.

In [41]:
best_model_output

Name,Datastore,Path on Datastore,Produced By PipelineRun
best_model_output,workspaceblobstore,azureml/a37b57e0-b4cf-455d-aa34-7b6f1a304ba1/model_data,6f86276d-c29a-4dbd-98dc-538743df3ab2


In [43]:
import pickle

with open(best_model_output_name, "rb" ) as f:
    best_model = pickle.load(f)
best_model

FileNotFoundError: [Errno 2] No such file or directory: 'best_model_output'

In [45]:
best_model.steps

NameError: name 'best_model' is not defined

### Test the Model
#### Load Test Data
For the test data, it should have the same preparation step as the train data. Otherwise it might get failed at the preprocessing step.

In [46]:
dataset_test = Dataset.Tabular.from_delimited_files(path='https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_test.csv')
df_test = dataset_test.to_pandas_dataframe()
df_test = df_test[pd.notnull(df_test['y'])]

y_test = df_test['y']
X_test = df_test.drop(['y'], axis=1)

#### Testing Our Best Fitted Model

We will use confusion matrix to see how our model works.

In [47]:
from sklearn.metrics import confusion_matrix
ypred = best_model.predict(X_test)
cm = confusion_matrix(y_test, ypred)

NameError: name 'best_model' is not defined

In [ ]:
# Visualize the confusion matrix
pd.DataFrame(cm).style.background_gradient(cmap='Blues', low=0, high=0.9)

## Publish and run from REST endpoint

Run the following code to publish the pipeline to your workspace. In your workspace in the portal, you can see metadata for the pipeline including run history and durations. You can also run the pipeline manually from the portal.

Additionally, publishing the pipeline enables a REST endpoint to rerun the pipeline from any HTTP library on any platform.


In [49]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Bankmarketing Train", description="Training bankmarketing pipeline", version="1.0")

published_pipeline


Name,Id,Status,Endpoint
Bankmarketing Train,72fd888a-86af-406c-a0a1-44b33b1020f5,Active,REST Endpoint


Authenticate once again, to retrieve the `auth_header` so that the endpoint can be used

In [50]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()



Get the REST url from the endpoint property of the published pipeline object. You can also find the REST url in your workspace in the portal. Build an HTTP POST request to the endpoint, specifying your authentication header. Additionally, add a JSON payload object with the experiment name and the batch size parameter. As a reminder, the process_count_per_node is passed through to ParallelRunStep because you defined it is defined as a PipelineParameter object in the step configuration.

Make the request to trigger the run. Access the Id key from the response dict to get the value of the run id.


In [51]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "pipeline-rest-endpoint"}
                        )

In [52]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception("Received bad response from the endpoint: {}\n"
                    "Response Code: {}\n"
                    "Headers: {}\n"
                    "Content: {}".format(rest_endpoint, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')
print('Submitted pipeline run: ', run_id)

Submitted pipeline run:  031fdc3c-6985-4056-82e8-904289e7c905


Use the run id to monitor the status of the new run. This will take another 10-15 min to run and will look similar to the previous pipeline run, so if you don't need to see another pipeline run, you can skip watching the full output.

In [53]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

KeyError: 'log_files'